**<h2>Importing necessary libraries</h2>** 


In [15]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch


**<h2>Defining directories</h2>**

In [16]:
# Define data directories
image_dir = '/home/mhpromit7473/Documents/ISIC_Dataset/images'
superpixel_dir = '/home/mhpromit7473/Documents/ISIC_Dataset/superpixels'
cleaned_images_dir = '/home/mhpromit7473/Documents/ISIC_Dataset/cleaned_images'
augmented_images_dir = '/home/mhpromit7473/Documents/ISIC_Dataset/augmented_images'
masks_dir = '/home/mhpromit7473/Documents/ISIC_Dataset/masks'

# Create directories if they don't exist
os.makedirs(cleaned_images_dir, exist_ok=True)
os.makedirs(augmented_images_dir, exist_ok=True)
os.makedirs(masks_dir, exist_ok=True)

**<h2>Transformation and Augmentation</h2>**

In [17]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Data augmentation transformations
aug_transform = transforms.Compose([
    transforms.RandomRotation(degrees=30),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

**<h2>Custom Dataset Class:</h2>**
**<h4>Create a custom dataset class to handle loading images and superpixel images</h4>**

In [18]:

# Custom dataset class
class SkinLesionDataset(Dataset):
    def __init__(self, image_dir, superpixel_dir=None, transform=None, augment=False):
        self.image_dir = image_dir
        self.superpixel_dir = superpixel_dir
        self.transform = transform
        self.augment = augment
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.images[idx])
        image = Image.open(image_path).convert("RGB")

        # Load superpixel image if available
        superpixel = None
        if self.superpixel_dir:
            superpixel_path = os.path.join(self.superpixel_dir, self.images[idx].replace('.jpg', '_superpixels.png'))
            if os.path.exists(superpixel_path):
                superpixel = Image.open(superpixel_path).convert("RGB")

        # Apply transformations
        if self.transform:
            image = self.transform(image)
            if superpixel is not None:
                superpixel = self.transform(superpixel)

        # Apply data augmentation if requested
        if self.augment:
            image = aug_transform(image)
            if superpixel is not None:
                superpixel = aug_transform(superpixel)

        return (image, superpixel) if superpixel is not None else image


**<h2>data loaders</h2>**

In [19]:
# Create the datasets and dataloaders
dataset = SkinLesionDataset(image_dir=image_dir, superpixel_dir=superpixel_dir, transform=transform)
augmented_dataset = SkinLesionDataset(image_dir=image_dir, superpixel_dir=superpixel_dir, transform=transform, augment=True)

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
augmented_dataloader = DataLoader(augmented_dataset, batch_size=16, shuffle=True)

**<h2>Image Preprocessing</h2>**

In [27]:
# Apply DullRazor algorithm and save cleaned images
for i, (images, superpixels) in enumerate(dataloader):
    # Convert tensor to NumPy array
    image_np = images.permute(0, 2, 3, 1).numpy()

    # Rescale and convert to uint8
    image_np = ((image_np + 1) * 127.5).astype(np.uint8)

    # Convert to BGR for OpenCV processing
    image_bgr = [cv2.cvtColor(img, cv2.COLOR_RGB2BGR) for img in image_np]

    # Apply DullRazor algorithm
    for j, img in enumerate(image_bgr):
        grayScale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
        blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
        bhg = cv2.GaussianBlur(blackhat, (3, 3), cv2.BORDER_DEFAULT)
        ret, mask = cv2.threshold(bhg, 15, 255, cv2.THRESH_BINARY)
        dst = cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA)
        cleaned_image = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
        cv2.imwrite(os.path.join(cleaned_images_dir, f'cleaned_image_{i*16+j}.png'), cleaned_image)

**<h3 style="color:#2F4F4F;">Save augmented images</h3>**

In [24]:
# Save augmented images
for filename in os.listdir(cleaned_images_dir):
    cleaned_image_path = os.path.join(cleaned_images_dir, filename)
    cleaned_image = Image.open(cleaned_image_path).convert("RGB")
    augmented_image = aug_transform(cleaned_image)
    
    # Convert tensor to NumPy array and save
    augmented_image_np = augmented_image.permute(1, 2, 0).byte().numpy()
    augmented_image_pil = Image.fromarray(augmented_image_np)
    augmented_image_pil.save(os.path.join(augmented_images_dir, filename))


In [28]:
# Function to create masks
def create_masks(image_dir, output_dir):
    for filename in os.listdir(image_dir):
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Apply thresholding to create a binary mask
        ret, mask = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Save the mask
        mask_path = os.path.join(output_dir, filename.replace('.png', '_mask.png'))
        cv2.imwrite(mask_path, mask)

In [30]:
# Create masks for cleaned images
create_masks(cleaned_images_dir, masks_dir)

# Create masks for augmented images
# create_masks(augmented_images_dir, masks_dir)